# Nextstrain

In [ ]:
# 1
## 下載數據 (Python 3)
!mkdir -p ~/Racmacs/nextstrain/myfile
%cd ~/Racmacs/nextstrain/myfile
!rm -rf results/
!mkdir -p results/

### 切換環境 (Image nextstain)

In [ ]:
# 2
## 建置metadata
## metadata for gisaid data, gisaid.fasta 
## Isolate name | Isolate ID | Segment | Collection date | Type
%cd ~/Racmacs/nextstrain/myfile
!augur parse \
    --sequences gisaid.fasta \
    --fields strain accession segment date virus \
    --output-sequences results/sequences.fasta \
    --output-metadata results/metadata.tsv

In [ ]:
# 3
## Index the Sequences
%cd ~/Racmacs/nextstrain/myfile
!augur index \
  --sequences results/sequences.fasta \
  --output results/sequence_index.tsv

In [ ]:
# 4
## Filter the Sequences
%cd ~/Racmacs/nextstrain/myfile
!augur filter \
  --sequences results/sequences.fasta \
  --sequence-index results/sequence_index.tsv \
  --metadata results/metadata.tsv \
  --output results/filtered.fasta \
  --min-length 1500

In [ ]:
# 5
## Align the Sequences
%cd ~/Racmacs/nextstrain/myfile
!augur align \
  --sequences results/filtered.fasta \
  --reference-sequence ~/Racmacs/nextstrain/avian-flu/config/reference_h5n1_ha.gb \
  --output results/aligned.fasta \
  --fill-gaps

In [ ]:
# 6
## Construct the Phylogeny
## http://etetoolkit.org/treeview/
%cd ~/Racmacs/nextstrain/myfile
!augur tree \
    --alignment results/aligned.fasta \
    --output results/tree_raw.nwk \
    --method iqtree \
    --nthreads 1

In [ ]:
# 7
## tree refine
%cd ~/Racmacs/nextstrain/myfile
!augur refine \
  --tree results/tree_raw.nwk \
  --alignment results/aligned.fasta \
  --metadata results/metadata.tsv \
  --output-tree results/tree.nwk \
  --output-node-data results/branch_lengths.json \
  --timetree \
  --coalescent const \
  --date-confidence \
  --date-inference marginal \
  --clock-filter-iqd 4

In [ ]:
# 8
## ancestral
%cd ~/Racmacs/nextstrain/myfile
!augur ancestral \
  --tree results/tree.nwk \
  --alignment results/aligned.fasta \
  --output-node-data results/nt_muts.json \
  --inference joint

In [ ]:
# 9
## translate
%cd ~/Racmacs/nextstrain/myfile
!augur translate \
  --tree results/tree.nwk \
  --ancestral-sequences results/nt_muts.json \
  --reference-sequence ~/Racmacs/nextstrain/avian-flu/config/reference_h5n1_ha.gb \
  --output-node-data results/aa_muts.json

In [ ]:
# 10
## traits
%cd ~/Racmacs/nextstrain/myfile
!augur traits \
  --tree results/tree.nwk \
  --metadata results/metadata.tsv \
  --output-node-data results/traits.json \
  --columns country \
  --confidence

In [ ]:
# 11
## export json
%cd ~/Racmacs/nextstrain/myfile
!augur export v2 \
  --tree results/tree.nwk \
  --metadata results/metadata.tsv \
  --node-data results/branch_lengths.json \
              results/traits.json \
              results/nt_muts.json \
              results/aa_muts.json \
  --colors ~/Racmacs/nextstrain/avian-flu/config/colors_h5n1.tsv \
  --lat-longs ~/Racmacs/nextstrain/avian-flu/config/lat_longs_h5n1.tsv \
  --auspice-config ~/Racmacs/nextstrain/avian-flu/config/auspice_config_h5n1.json \
  --output auspice/avian-flu.json

In [ ]:
# 12
## 建立數據視圖
%cd ~/Racmacs/nextstrain/myfile
!cp /work/c00cjz002/jupyter/config.json .
!auspice build --extend config.json

In [ ]:
## 服務器
%cd ~/Racmacs/nextstrain/myfile
!export PORT=12300; auspice view --datasetDir auspice